In [1]:
import cv2
import os
import glob
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from models import DnCNN
from utils import *

ModuleNotFoundError: No module named 'models'

In [ ]:
# Please modified the following codes for your own use
modelPath = 'finalnew/beta10/'+'model_29.pth'
f = glob.glob(os.path.join('data', 'Set12', '02.png'))
NoisyPictureName = 'beta10_TDist_Noisy02.png'
CleanPictureName = 'beta10_TDist_Clean02.png'

In [ ]:

def getSize(noise):
    k = 1
    for i in range(len(noise.size())):
        k *= noise.size()[i]
    return k

def normalize(data):
    return data/255.

def unnormalize(data):
    return data*255.
def getDenoisedImage(modelPath, file, NoisyPictureName, CleanPictureName)
    ## Translate raw image
    Img = cv2.imread(f[0])
    Img = normalize(np.float32(Img[:,:,0]))
    Img = np.expand_dims(Img, 0)
    Img = np.expand_dims(Img, 1)
    ISource = torch.Tensor(Img)

    ## Add noise
    flatSize = getSize(ISource)
    noise = torch.FloatTensor(t.rvs(5.2, size=flatSize))
    noise = noise.view(ISource.size())
    INoisy = ISource + noise
    INoisyTBSave = INoisy

    ###Save Noisy picture
    INoisyTBSave = INoisyTBSave[0,0,:,:]
    INoisyTBSave = unnormalize(np.float32(INoisyTBSave))
    INoisyTBSave = np.expand_dims(INoisyTBSave, 2)
    cv2.imwrite(NoisyPictureName,INoisyTBSave)

    ## Load Model
    net = DnCNN(channels=1, num_of_layers=17)
    device_ids = [0]
    model = nn.DataParallel(net, device_ids=device_ids).cuda()
    model.load_state_dict(torch.load(modelPath))
    model.eval()

    ## send noisy image to model
    ISource, INoisy = Variable(ISource.cuda()), Variable(INoisy.cuda())
    with torch.no_grad(): # this can save much memory
        Out = torch.clamp(INoisy-model(INoisy), 0., 1.)

    ### Save clean picture
    Out = Out[0,0,:,:]
    Out = unnormalize(np.float32(Out.cpu()))
    Out = np.expand_dims(Out, 2)
    cv2.imwrite(CleanPictureName,Out)
    return Out